In [519]:
import pandas as pd
import numpy as np
import math

In [520]:
#print(asks_df.index)
#print(min(asks_df.index))
#print(max(asks_df.index))

# RESUMEN

In [521]:
asks_df = pd.read_csv("asks_df.csv")
asks_df.sort_index(ascending=False, inplace=True)
asks_df.set_index("price", inplace=True)
bids_df = pd.read_csv("bids_df.csv")
bids_df.sort_index(ascending=False, inplace=True)
bids_df.set_index("price", inplace=True)

## 1. Tener los múltiplos a usar y la precisión para redondear

In [522]:
i1 = 0.01
i2 = 0.001

In [523]:
precision = 4

## 2. Obtenemos la lista de precios a través de un panda Series

In [524]:
asks_numbers_series = pd.Series(asks_df.index)
bids_numbers_series = pd.Series(bids_df.index)

## 3. Obtenemos los valores máximos y mínimos de los precios para cada DataFrame

In [525]:
minimo_asks = round(asks_numbers_series.min(), precision)
maximo_asks = round(asks_numbers_series.max(), precision)
print(minimo_asks)
print(maximo_asks)

0.8003
0.9006


In [526]:
minimo_bids = round(bids_numbers_series.min(), precision)
maximo_bids = round(bids_numbers_series.max(), precision)
print(minimo_bids)
print(maximo_bids)

0.7001
0.8002


## 4. Obtenemos valores iniciales y finales de los rangos para ASKS y BIDS

- Como debemos redondear en base a la cantidad de decimales que tenga el múltiplo, haremos:
    - Para el mínimo, encontrar cuántas veces cabe el múltiplo en el mínimo
    - Este valor lo redondeamos hacia el entero inferior con math.ceil()
    - Regresamos a los decimales, multiplicando con el múltiplo

In [527]:
def crear_rango_minimo(minimo: float, multiplo: float) -> float:
    global precision
    div = minimo / multiplo
    min_entero = math.floor(div)
    return round(min_entero * multiplo, precision)

Para crear el valor máximo del rango, haremos:
- dividimos el valor máximo del rango entre el múltiplo
- el resultado lo redondeamos al entero superior
- regresamos a los decimales multiplicando por el múltiplo
- En el caso del máximo debemos sumarle al resultado el múltiplo porque el rango es abierto a la derecha (máximo)

In [528]:
def crear_rango_maximo(maximo: float, multiplo:float) -> float:
    div = maximo / multiplo
    max_entero = math.ceil(div)
    return round((max_entero * multiplo) + 0.01, precision)

Entonces, para ASKS, tenemos:

In [529]:
rango_min_asks_i1 = crear_rango_minimo(minimo_asks, i1)
rango_min_asks_i1

0.8

In [530]:
rango_max_asks_i1 = crear_rango_maximo(maximo_asks, i1)
rango_max_asks_i1

0.92

Para BIDS, tenemos

In [531]:
rango_min_bids_i1 = crear_rango_minimo(minimo_bids, i1)
rango_min_bids_i1

0.7

In [532]:
rango_max_bids_i1 = crear_rango_maximo(maximo_bids, i1)
rango_max_bids_i1

0.82

- PARA RANGO i2

In [543]:
rango_min_asks_i2 = crear_rango_minimo(minimo_asks, i2)
rango_max_asks_i2 = crear_rango_maximo(maximo_asks, i2)
rango_min_bids_i2 = crear_rango_minimo(minimo_bids, i2)
rango_max_bids_i2 = crear_rango_maximo(maximo_bids, i2)

## 5. Creamos los rangos de precios con np.arange()

Estos rangos lo llamaremos bins, ya que los usaremos después para usarlos en el dataframe

### Asks

In [533]:
asks_bins_i1 = np.arange(rango_min_asks_i1, rango_max_asks_i1, i1)
asks_bins_i1

array([0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.9 ,
       0.91])

### Bids

In [534]:
bids_bins_i1 = np.arange(rango_min_bids_i1, rango_max_bids_i1, i1)
bids_bins_i1

array([0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8 ,
       0.81])

### Para rango i2

In [544]:
asks_bins_i2 = np.arange(rango_min_asks_i2, rango_max_asks_i2, i2)
bids_bins_i2 = np.arange(rango_min_bids_i2, rango_max_bids_i2, i2)

# 6. Creamos los rangos de precios con pd.cut()

- En el DataFrame creamos la nueva columna que será el rango el cual nos dirá a que rango pertenece cada precio
- Esteo lo logramos con pd.cut() sobre los índices del DataFrame (porque son los precios) y usando los bins creados

In [535]:
asks_df['rango_i1'] = pd.cut(asks_df.index, asks_bins_i1)
asks_df

,quantity,rango_i1
price,,
0.9006,674.0,"(0.9, 0.91]"
0.9005,335.0,"(0.9, 0.91]"
0.9004,175.0,"(0.9, 0.91]"
0.9003,501.0,"(0.9, 0.91]"
0.9002,5824.0,"(0.9, 0.91]"
...,...,...
0.8007,101220.0,"(0.8, 0.81]"
0.8006,80521.0,"(0.8, 0.81]"
0.8005,78522.0,"(0.8, 0.81]"


In [536]:
bids_df['rango_i1'] = pd.cut(bids_df.index, bids_bins_i1)
bids_df

,quantity,rango_i1
price,,
0.8002,15888.0,"(0.8, 0.81]"
0.8001,42942.0,"(0.8, 0.81]"
0.8000,66902.0,"(0.79, 0.8]"
0.7999,50941.0,"(0.79, 0.8]"
0.7998,66982.0,"(0.79, 0.8]"
...,...,...
0.7005,1373.0,"(0.7, 0.71]"
0.7004,1515.0,"(0.7, 0.71]"
0.7003,27.0,"(0.7, 0.71]"


### Para rango i2

In [545]:
asks_df['rango_i2'] = pd.cut(asks_df.index, asks_bins_i2)
bids_df['rango_i2'] = pd.cut(bids_df.index, bids_bins_i2)

# 7. Agrupamos los datos por el rango y sumamos columna 'quantity'

- Al usar groupby, lo que tendremos es un objeto DataFrame por lo que no podremos verlos así de fácil
- para poder ver su contenido podemos crear una función que imprima cada elemento. Ejemplificamos creando la función print_group, haciendo a alusión a un grupo de datos y al DataFrame agrupado le hacemos apply y usamos esta función para que imprima los registros
- Ya que tenemos los registros agrupados por el rango, hacemos la suma de las cantidades. Para esto aplicamos el método sum() sobre la columna quantitys y, a su vez, sobre el DataFrame agrupado por rangos

In [537]:
def print_group(group):
    print(group)

In [538]:
def sumar_cantidades(df: pd.DataFrame, rango:str) -> pd.DataFrame:
    grouped = df.groupby('rango_'+rango)
    #asks_grouped.apply(print_group)
    return grouped['quantity'].sum()

In [539]:
suma_cantidades_asks_i1 = sumar_cantidades(asks_df, 'i1')
suma_cantidades_asks_i1

rango_i1
(0.8, 0.81]     3474525.0
(0.81, 0.82]    2240488.0
(0.82, 0.83]    1197949.0
(0.83, 0.84]     994549.0
(0.84, 0.85]     944127.0
(0.85, 0.86]     540548.0
(0.86, 0.87]     305108.0
(0.87, 0.88]     372564.0
(0.88, 0.89]     294532.0
(0.89, 0.9]      354691.0
(0.9, 0.91]        7533.0
Name: quantity, dtype: float64

In [540]:
suma_cantidades_bids_i1 = sumar_cantidades(bids_df, 'i1')
suma_cantidades_bids_i1

rango_i1
(0.7, 0.71]      391013.0
(0.71, 0.72]     334497.0
(0.72, 0.73]     796789.0
(0.73, 0.74]     406827.0
(0.74, 0.75]     877647.0
(0.75, 0.76]     495403.0
(0.76, 0.77]     902942.0
(0.77, 0.78]    1533820.0
(0.78, 0.79]    1245011.0
(0.79, 0.8]     3891873.0
(0.8, 0.81]       58830.0
Name: quantity, dtype: float64

### Para rango i2

In [546]:
suma_cantidades_asks_i2 = sumar_cantidades(asks_df, 'i2')
suma_cantidades_bids_i2 = sumar_cantidades(bids_df, 'i2')

# Obtenemos el índice (Precio) con mayor volumen

Para esto usaremos el DataFrame obtenido anteriormente

In [541]:
suma_cantidades_asks_i1.idxmax().right

0.81

In [542]:
suma_cantidades_bids_i1.idxmax().right

0.8

In [547]:
suma_cantidades_asks_i2.idxmax().right

0.802

In [548]:
suma_cantidades_bids_i2.idxmax().right

0.799